In [3]:
from cit_images import batch_predict, read_boxes64, read_boxes
from tf.serving import predict, multipredict

In [4]:
read_boxes(images_path='.images')

[('.images/boxes/boxes-0-0.jpg', '.images/boxes/boxes-1-0.jpg'),
 ('.images/boxes/boxes-0-1.jpg', '.images/boxes/boxes-1-1.jpg'),
 ('.images/boxes/boxes-0-2.jpg', '.images/boxes/boxes-1-2.jpg'),
 ('.images/boxes/boxes-0-3.jpg', '.images/boxes/boxes-1-3.jpg'),
 ('.images/boxes/boxes-0-4.jpg', '.images/boxes/boxes-1-4.jpg'),
 ('.images/boxes/boxes-0-5.jpg', '.images/boxes/boxes-1-5.jpg'),
 ('.images/boxes/boxes-0-6.jpg', '.images/boxes/boxes-1-6.jpg'),
 ('.images/boxes/boxes-0-7.jpg', '.images/boxes/boxes-1-7.jpg')]

In [2]:
await multipredict(read_boxes64(images_path='.images'), host='http://localhost/')

Right(value=[([('e4', -0.187645733), ('e4', -2.03857732), ('e5', -5.38002157), ('ed4', -5.4862504), ('de4', -5.90797043), ('d4', -5.94355), ('Ae4', -6.26216507), ('e3', -6.26460314), ('Re4', -6.38398266), ('c4', -6.42287493), ('e6', -6.52720642), ('Te4', -6.62037849), ('x4', -6.7190876), ('ed4', -6.84687185), ('ce4', -6.91570187), ('ef4', -7.18795204), ('e5', -7.23095608), ('Ce4', -7.27822828), ('De4', -7.35388374), ('e7', -7.52508688), ('e1', -7.60124779), ('e8', -7.67382145), ('fe4', -7.68926764), ('ec4', -7.72251797), ('f4', -7.86132145)], [('e4', -0.141589686), ('e4', -2.35486913), ('e5', -5.24382877), ('c4', -5.4035759), ('d4', -6.02367878), ('ed4', -6.24456215), ('e3', -6.30107641), ('e6', -6.37659121), ('ce4', -6.44709778), ('x4', -6.58544731), ('e7', -6.77404118), ('Ae4', -7.0326848), ('e2', -7.11654758), ('Ce4', -7.13149834), ('de4', -7.15152788), ('Re4', -7.16754055), ('Te4', -7.21641922), ('b4', -7.39393711), ('e5', -7.45711231), ('a4', -7.53002357), ('f4', -7.55076027), ('e